In [9]:
import sys
print (sys.version)

3.7.2 (v3.7.2:9a3ffc0492, Dec 24 2018, 02:44:43) 
[Clang 6.0 (clang-600.0.57)]


In [10]:
#https://stackoverflow.com/questions/47944782/r-data-scraping-crawling-with-dynamic-multiple-urls
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [11]:
import locale, datetime #for stringparsing in german
locale.setlocale(locale.LC_ALL, "de_de")

'de_de'

**Extracting the index leading to mandate**

In [12]:
#highlight_docid=atf%3A%2F%2F95-IV-162
queryKeyword = "mord"
queryRank = "1"
#bgerQuery2 = "https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=1954&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words="+ queryKeyword + "&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank="+queryRank+"&highlight_docid=atf%3A%2F%2F95-IV-162%3Ade&number_of_ranks=945&azaclir=clir"

In [13]:
bgerQuery2 = 'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=simple_query&query_words='+ queryKeyword +'&lang=de&top_subcollection_clir=bge&from_year=1954&to_year=2019'

In [14]:
bgerQuery2

'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=simple_query&query_words=mord&lang=de&top_subcollection_clir=bge&from_year=1954&to_year=2019'

In [15]:
#check if content was loaded properly
bgerQuery2 = requests.get(bgerQuery2)
bgerQuery2

<Response [200]>

In [16]:
#bgerQuery2.content

In [17]:
#soup = BeautifulSoup(page.content, 'html.parser')

In [18]:
bgerSoup = BeautifulSoup(bgerQuery2.content, 'html.parser')

In [19]:
bgerSoup


<!DOCTYPE html>

<html lang="de">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width initial-scale=1" name="viewport"/>
<link href="/modules/tfl-internet-templates/images/favicon.ico" rel="icon"/>
<title>Eurospider Suche: mord</title>
<link href="https://www.bger.ch/ext/eurospider/live/de/php/clir/http/css/master.css" rel="stylesheet" title="Eurospider Default Screen Style" type="text/css"/>
<link href="/modules/tfl-internet-templates/css/styles.css" media="screen" rel="stylesheet" type="text/css"/>
<style media="print" type="text/css">
        @media print{
            .body .size1of3{
                display: none;
            }
            .head .grid_left{
                display: none;
            }
            .access-a, .form-search, .define-admin {
                display: none !important;
            }
            body{
                font-family: Arial, sans-serif;
                font-size: 12px !important;
       

In [20]:
links = bgerSoup.find_all(class_='rank_title')
linksList = []
for a in links:
    linksList.append(a.find('a')['href'])
    #print(a)
print(linksList[0])

https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=1954&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words=mord&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank=1&highlight_docid=atf%3A%2F%2F95-IV-162%3Ade&number_of_ranks=73&azaclir=clir


**Make a new query with Linkslist (only based on first entry)**

In [21]:
mBger = requests.get(linksList[0])

In [22]:
#mBger.content

In [23]:
mBger = BeautifulSoup(mBger.content, 'html.parser')

In [24]:
#mBger

**display children / find correct html elements**

In [25]:
len(list(mBger.children))
#find body
body = list(mBger.children)[12]
len(list(body.children))
#body

33

**get all required variables from verdict**

**dateparser for string dates**

In [26]:
text2 = "41. Urteil des Kassationshofes vom 5. Dezember 1969 i.S. Staatsanwaltschaft des Kantons Aargau gegen Brunner."

def parseDate(text):
    pattern = re.compile(r"""
        \d\d?   # one or two digits
        \.?
        \s?      # \s for space - one space after
        [a-z,ä,ö,ü]+  # at least one+ ascii letters (ignore case is use)set of characters that you wish to match / since + means ‘one or more repetitions’
        \s?      # one space after
        \d{4}   # four decimal digits = \d (year)
    """,re.IGNORECASE|re.VERBOSE)
    if (pattern.search(text)!= None):
        #print(pattern.search(text).group(), type(pattern.search(text).group()))
        dateStr = pattern.search(text).group()
        dateStr = dateStr.replace(".", "")
        dateStr = datetime.datetime.strptime(dateStr, "%d %B %Y")
        #dateStr = dateStr.strftime('%m/%d/%Y')
        return dateStr
    else:
        return 0 #make 0 if date can't be found

In [27]:
#ID of mandate
mID = body.find_all('div', class_="center pagebreak")[0].get_text()[:13]
#mandID

#referenced mandates
#without link
mRefs = body.find_all(class_="bgeref_err")
#with link
mRefsLinked = body.find_all(class_="bgeref_id")

#date:
mDate = body.find_all(class_="paraatf")[0].get_text()
#mDate = parseDate(mDate)
mDate = parseDate(mDate)
mDate

#links of liked reference mandates
linksListref = []
for a in mRefsLinked:
    linksListref.append(a['href'])

In [28]:
type(mDate)

datetime.datetime

In [29]:
def listing(elementlist):
    ar = []
    for item in elementlist:
        print(item.get_text())
        ar.append(item.get_text())
    print(ar)
    return ar

In [30]:
mRefs = listing(mRefs)
mRefsLinked = listing(mRefsLinked)

BGE 70 IV 7
['BGE 70 IV 7']
BGE 80 IV 238
BGE 80 IV 240
BGE 82 IV 8
BGE 87 IV 115
BGE 82 IV 6
BGE 82 IV 10
['BGE 80 IV 238', 'BGE 80 IV 240', 'BGE 82 IV 8', 'BGE 87 IV 115', 'BGE 82 IV 6', 'BGE 82 IV 10']


In [31]:
"id", mID, "referenced mandates", mRefs, "referenced linked mandates", mRefsLinked, "date", mDate

('id',
 'BGE 95 IV 162',
 'referenced mandates',
 ['BGE 70 IV 7'],
 'referenced linked mandates',
 ['BGE 80 IV 238',
  'BGE 80 IV 240',
  'BGE 82 IV 8',
  'BGE 87 IV 115',
  'BGE 82 IV 6',
  'BGE 82 IV 10'],
 'date',
 datetime.datetime(1969, 12, 5, 0, 0))

**Save variables in dynamic dictionary**

In [34]:
#l=['mID','mRefs','mRefsLinked','links','date']
#objs = [mID,mRefs, mRefsLinked, linksListref, mDate]
l=['mID','mRefs','mRefsLinked','date']
objs = [mID,mRefs, mRefsLinked, mDate]
mDict = {}
for i, val in enumerate(l):
    k= objs[i]
    mDict[val]=k
print(mDict, type(mDict))

{'mID': 'BGE 95 IV 162', 'mRefs': ['BGE 70 IV 7'], 'mRefsLinked': ['BGE 80 IV 238', 'BGE 80 IV 240', 'BGE 82 IV 8', 'BGE 87 IV 115', 'BGE 82 IV 6', 'BGE 82 IV 10'], 'date': datetime.datetime(1969, 12, 5, 0, 0)} <class 'dict'>


In [33]:
#https://stackoverflow.com/questions/13575090/construct-pandas-dataframe-from-items-in-nested-dictionary
#pd.DataFrame.from_dict(mDict, orient='index')